# Aula 04 - Avaliação de Sistemas de Recomendação - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (1).gz
x dataset/
x dataset/tags_sample.csv
x dataset/.DS_Store
x dataset/movies_sample.csv
x dataset/genome-tags.csv
x dataset/ml-youtube.csv
x dataset/genome-scores.csv
x dataset/ratings_sample.csv


In [14]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,359,7481,4.0,Enemy Mine (1985)
2,422,7481,3.0,Enemy Mine (1985)
3,424,7481,3.0,Enemy Mine (1985)
4,744,7481,3.0,Enemy Mine (1985)
...,...,...,...,...
190616,131651,118606,3.5,Dots (1940)
190617,134567,118606,3.5,Dots (1940)
190618,131651,96257,1.0,Heroic Purgatory (Rengoku eroica) (1970)
190619,131651,98337,4.0,97 Percent True (2008)


In [15]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [16]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [17]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [18]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

### Divisão da base em treino e teste

In [19]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

### Exemplos de utilização do CaseRecommender

In [21]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 3.779151 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.644618 sec
Eval:: MAE: 0.736439 RMSE: 0.966548 
[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 3.765856 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.690869 sec
Eval:: MAE: 0.762727 RMSE: 1.008645 


***Exercício 01:*** Verifique o efeito no RSME ao aumentar o número de vizinhos do algoritmo ItemAttributeKNN com usando gêneros. Explique.

In [ ]:
# TODO

***Exercício 02:*** Um importante aspecto que pode ser avaliado em Sistemas de Recomendação é a diversidade da lista de recomendações. A métrica Intra-List Similarity (ILS) aplica uma função de similaridade (e.g. Cosseno, Jaccard, Pearson, etc.) entre todos os pares de itens da lista de recomendação, usando seus metadados como gêneros, tags, etc. Mais detalhes sobre essa métrica podem ser encontrados em: https://grouplens.org/site-content/uploads/Improving-WWW-20051.pdf

***a)*** Implemente uma função que calcula a ILS de uma lista de recomendação para um único usuário. Utilize os gêneros de filmes.

In [ ]:
# TODO

***b)*** Utilize a função que implementou no item (a) para calcular a ILS de todos os usuários da base de dados. Utilize as recomendações geradas pelo algoritmo BPR MF do CaseRecommender.

In [ ]:
# TODO